# Document Splitting

Discover the best practices and considerations for splitting data.

# Setup

## Python

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from dotenv import load_dotenv, find_dotenv
import os
import openai
# import sys
# sys.path.append('../..')

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

# Splitting

## Character Text Splitter

In [ ]:
chunk_size = 26
chunk_overlap = 4

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

## Text 1

- Why doesn't this split the string below?

In [ ]:
text1 = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
r_splitter.split_text(text1)

- ['abcdefghijklmnopqrstuvwxyz']

## Text 2

In [ ]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

In [ ]:
r_splitter.split_text(text2)

- ['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']


## Text 3

In [ ]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [ ]:
r_splitter.split_text(text3)

- ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']




In [ ]:
c_splitter.split_text(text3)

- ['a b c d e f g h i j k l m n o p q r s t u v w x y z']

## CharacterTextSplitter

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=' '
)

In [ ]:
c_splitter.split_text(text3)

- ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

# Recursive Splitting Details

## RecursiveCharacterTextSplitter {.smaller}

- `RecursiveCharacterTextSplitter` is recommended for generic text.

In [ ]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

## Define splitter

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator=' '
)

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

## Character Splitter output

In [ ]:
c_splitter.split_text(some_text)

- ['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

## Recursive Splitter output


In [ ]:
r_splitter.split_text(some_text)

- ["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

## Adapt splitter 1 {.smaller}

- Let's reduce the chunk size a bit and add a period to our separators:

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)

In [ ]:
r_splitter.split_text(some_text)

- ["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related",
 '. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns',
 '. Carriage returns are the "backslash n" you see embedded in this string',
 '. Sentences have a period at the end, but also, have a space.and words are separated by space.']

## Adapt splitter 2 {.smaller}

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

In [ ]:
r_splitter.split_text(some_text)

- ["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related.",
 'For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns.',
 'Carriage returns are the "backslash n" you see embedded in this string.',
 'Sentences have a period at the end, but also, have a space.and words are separated by space.']

# Split a Document

## Load PDF

In [ ]:
loader = PyPDFLoader("../docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

## Define splitter

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

## Split document

In [ ]:
docs = text_splitter.split_documents(pages)

## Inspect data

In [ ]:
len(docs)

- 77



In [ ]:
len(pages)

- 22

## Inspect data

In [ ]:
print(docs[0].page_content[300:800])

- my name's  Andrew Ng and I'll be instru ctor for this class. And so
I personally work in machine learning, and I' ve worked on it for about 15 years now, and
I actually think that machine learning is th e most exciting field of all the computer
sciences. So I'm actually always excited about  teaching this class. Sometimes I actually
think that machine learning is not only the most exciting thin g in computer science, but
the most exciting thing in all of human e ndeavor, so maybe a little b

# Split Notion data

## Load data

In [ ]:
loader = NotionDirectoryLoader("../docs/Notion_DB")
notion_db = loader.load()

## Define splitter

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

## Split document

In [ ]:
docs = text_splitter.split_documents(notion_db)

## Inspect data

In [ ]:
len(notion_db)

- 1



In [ ]:
len(docs)

- 2


## Inspect data {smaller}

In [ ]:
print(docs[0].page_content)

```markdown
# Getting Started
👋 Welcome to Notion!
Here are the basics:
- [ ]  Click anywhere and just start typing
- [ ]  Hit `/` to see all the types of content you can add - headers, videos, sub pages, etc.
    
    [Example sub page](https://www.notion.so/Example-sub-page-92f63253929d456bbf12cd696e21e045?pvs=21)
    
- [ ]  Highlight any text, and use the menu that pops up to **style** *your* ~~writing~~ `however` [you](https://www.notion.so/product) like
- [ ]  See the `⋮⋮` to the left of this checkbox on hover? Click and drag to move this line
- [ ]  Click the `+ New Page` button at the bottom of your sidebar to add a new page
- [ ]  Click `Templates` in your sidebar to get started with pre-built pages
- This is a toggle block. Click the little triangle to see more useful tips!
    - [Template Gallery](https://www.notion.so/181e961aeb5c4ee6915307c0dfd5156d?pvs=21): More templates built by the Notion community

```

# Token splitting

## Basics

- We can also split on token count explicity, if we want

- This can be useful because LLMs often have context windows designated in tokens

- Tokens are often ~4 characters.


## TokenTextSplitter 1

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [ ]:
text1 = "foo bar bazzyfoo"

In [ ]:
text_splitter.split_text(text1)

- ['foo', ' bar', ' b', 'az', 'zy', 'foo']

## TokenTextSplitter 2 {.smaller}


In [ ]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
docs[0]

- Document(page_content='MachineLearning-Lecture01  \n', metadata={'source': '../docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0})



In [ ]:
pages[0].metadata

- {'source': '../docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}

# Context Aware Splitting with Markdown

## Basics

- Chunking aims to keep text with common context together.

- A text splitting often uses sentences or other delimiters to keep related text together but many documents (such as Markdown) have structure (headers) that can be explicitly used in splitting.

- We can use `MarkdownHeaderTextSplitter` to preserve header metadata in our chunks

## Markdown example

In [ ]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n
## Chapter 2\n\n \
Hi this is Molly"""

## Headers to split on

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

## MarkdownHeaderTextSplitter

In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

## Split text

In [ ]:
md_header_splits = markdown_splitter.split_text(markdown_document)

In [ ]:
md_header_splits[0]

- Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'})



In [ ]:
md_header_splits[1]

- Document(page_content='Hi this is Lance', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'})

# Splitting Notion Markdown

## Load data


In [ ]:
loader = NotionDirectoryLoader("../docs/Notion_DB")
docs = loader.load()

## Join data {.smaller}

In [ ]:
txt = ' '.join([d.page_content for d in docs])
txt

- '# Getting Started\n\n👋 Welcome to Notion!\n\nHere are the basics:\n\n- [ ]  Click anywhere and just start typing\n- [ ]  Hit `/` to see all the types of content you can add - headers, videos, sub pages, etc.\n    \n    [Example sub page](https://www.notion.so/Example-sub-page-92f63253929d456bbf12cd696e21e045?pvs=21)\n    \n- [ ]  Highlight any text, and use the menu that pops up to **style** *your* ~~writing~~ `however` [you](https://www.notion.so/product) like\n- [ ]  See the `⋮⋮` to the left of this checkbox on hover? Click and drag to move this line\n- [ ]  Click the `+ New Page` button at the bottom of your sidebar to add a new page\n- [ ]  Click `Templates` in your sidebar to get started with pre-built pages\n- This is a toggle block. Click the little triangle to see more useful tips!\n    - [Template Gallery](https://www.notion.so/181e961aeb5c4ee6915307c0dfd5156d?pvs=21): More templates built by the Notion community\n    - [Help & Support](https://www.notion.so/e040febf70a94950b8620e6f00005004?pvs=21): ****Guides and FAQs for everything in Notion\n    - Stay organized with your sidebar and nested pages:\n        \n        ![Getting%20Started%2095e5ecbe48c44e408ef09fed850fbd40/infinitehierarchynodither.gif](https://github.com/kirenz/lab-langchain-rag/blob/main/code/Getting%20Started%2095e5ecbe48c44e408ef09fed850fbd40/infinitehierarchynodither.gif?raw=1)\n        \n    \n\nSee it in action:\n\n[1 minute](https://youtu.be/TL_N2pmh9O0)\n\n1 minute\n\n[4 minutes](https://youtu.be/FXIrojSK3Jo)\n\n4 minutes\n\n[2 minutes](https://youtu.be/2Pwzff-uffU)\n\n2 minutes\n\n[2 minutes](https://youtu.be/O8qdvSxDYNY)\n\n2 minutes\n\nVisit our [YouTube channel](http://youtube.com/c/notion) to watch 50+ more tutorials\n\n👉**Have a question?** Click the `?` at the bottom right for more guides, or to send us a message.'

## Define Splitter

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

## Split text

In [ ]:
md_header_splits = markdown_splitter.split_text(txt)

## Output {.smaller}

In [ ]:
md_header_splits[0]